In [14]:
from minio.error import S3Error
import fitz  # PyMuPDF
from datetime import datetime
from io import BytesIO
import pandas as pd
import re
import psycopg2
import import_ipynb
from Utils import conectar_minio, generar_ruta_fecha,extraer_archivo_minio, insert_into_info_lab, cargar_dfpaciente, agregar_fuente, agregar_observacion
import os
from dotenv import load_dotenv

In [15]:
# Cargar variables del archivo .env
load_dotenv()

True

In [16]:
minio_client = conectar_minio(
    endpoint=os.getenv("MINIO_ENDPOINT"),
    access_key=os.getenv("MINIO_ACCESS_KEY"),
    secret_key=os.getenv("MINIO_SECRET_KEY"),
    secure=False  # Usar HTTPS
)

Conexión exitosa a MinIO en localhost:9000
Buckets disponibles: ['yachay', 'yachay-bronze', 'yachay-landing']


In [17]:
ruta_fecha = generar_ruta_fecha() 

In [18]:
 #3. Extraer un archivo Excel/CSV/Parquet
df_datos = extraer_archivo_minio(
    minio_client=minio_client,
    bucket_name="yachay-bronze",
    ruta_archivo=f"laboratorios/{ruta_fecha}/laboratorio San Pedro/San Pedro.csv",
    tipo_archivo="csv"
)
print(df_datos)  # Mostrar las primeras filas del DataFrame extraído

           ADMISION TIPO ID  NOHISTORIA                          PACIENTE  \
0           1285375      CC    12865021         GOMEZ SALAS VICENTE RAMON   
1           1274217      CC     5256941        ARTEAGA ROSERO JOSE MARINO   
2           1274357      CC    27093153  DELGADO CABRERA ROMELIA TRINIDAD   
3           1273941      CC    27387098  OBANDO ARTEAGA CENEIDA GUADALUPE   
4           1274230      CC    27079284         BENAVIDES  LAURA VICTORIA   
...             ...     ...         ...                               ...   
11790       1415305      CC    87220025   BENAVIDES ORDOÑEZ ANTONIO MARIA   
11791       1415408      CC    59799383   RODRIGUEZ CEBALLOS MERY EULALIA   
11792       1415560      CC  1086358558          GOMEZ DIAZ KAREN MELISSA   
11793           NaN     NaN         NaN                               NaN   
11794  05/feb./2024     NaN         NaN                               NaN   

      FECHA NACIMIENTO       SEXO  \
0           1953-04-05  MASCULINO   
1

In [ ]:
def transformar_df_lab_SanPedro(df_datos):
    """
    Transforma un DataFrame del laboratorio Jimenes a la estructura de id_infolab
    
    Parámetros:
    - df_datos: DataFrame con los campos originales del laboratorio
    
    Retorna:
    - DataFrame transformado con la estructura deseada
    """
    
    # Crear copia para no modificar el original
    df_transformado = df_datos.copy()
    
    # Eliminar espacios en los nombres de las columnas
    df_transformado.columns = df_transformado.columns.str.strip()
    
    # Mapeo de campos
    mapeo_campos = {
        'FECHA REALIZADO': 'fecha_ingreso',
        'FECHA RECEPCION': 'fecha_toma_muestra',
        'EMPRESA': 'eps',
        'MEDICO ORDENA': 'medico_ordenante',
        'PROCEDIMIENTO': 'muestra_remitida',
        'DESCRIPCION MACRO': 'descripcion_macroscopica',
        'DESCRIPCION MICRO': 'descripcion_microscopica',
        'DX HISTOPATOLOGICO': 'diagnostico',
        'NOHISTORIA': 'identificacion',
        'PACIENTE': 'nombre1',
        'TIPO ID': 'tipo_de_identificacion',
        'FECHA NACIMIENTO': 'fecha_nacimiento',
        'SEXO': 'sexo',
        'ATENCION':'servicio'
    }
    
    # Renombrar columnas
    df_transformado = df_transformado.rename(columns=mapeo_campos)
    
    # Crear campo id_paciente concatenando identificacion y tipo_de_identificacion
    df_transformado['id_paciente'] = (
        df_transformado['tipo_de_identificacion'].astype(str) + 
        df_transformado['identificacion'].astype(str)
    )
    
    return df_transformado

In [20]:
def estandarizar_sexo(df):
    """
    Versión simplificada que solo convierte:
    - "FEMENINO" exacto a "F"
    - "MASCULINO" exacto a "M"
    """
    df_estandarizado = df.copy()
    
    if 'sexo' in df_estandarizado.columns:
        df_estandarizado['sexo'] = df_estandarizado['sexo'].replace({
            'FEMENINO': 'F',
            'MASCULINO': 'M'
        })
    
    return df_estandarizado

In [21]:
df = transformar_df_lab_SanPedro(df_datos)
df = estandarizar_sexo(df)
df = agregar_fuente(df, '002')
df = agregar_observacion(df)
print(df.columns)
print(df.head())  # Mostrar las columnas del DataFrame transformado

Index(['ADMISION', 'tipo_de_identificacion', 'identificacion', 'nombre1',
       'fecha_nacimiento', 'sexo', 'entidad', 'ATENCION', 'PABELLON', 'CODIGO',
       'muestra_remitida', 'servicio', 'No. PATOLOGIA', 'fecha_toma_muestra',
       'fecha_ingreso', 'DIAS', 'descripcion_macroscopica',
       'descripcion_microscopica', 'diagnostico', 'id_paciente', 'fuente',
       'comentarios', 'observacion'],
      dtype='object')
  ADMISION tipo_de_identificacion identificacion  \
0  1285375                     CC       12865021   
1  1274217                     CC        5256941   
2  1274357                     CC       27093153   
3  1273941                     CC       27387098   
4  1274230                     CC       27079284   

                            nombre1 fecha_nacimiento sexo  \
0         GOMEZ SALAS VICENTE RAMON       1953-04-05    M   
1        ARTEAGA ROSERO JOSE MARINO       1944-11-14    M   
2  DELGADO CABRERA ROMELIA TRINIDAD       1957-11-28    F   
3  OBANDO ARTEAG

In [22]:
connection_params = {
    'dbname': os.getenv("DB_NAME"),
    'user': os.getenv("DB_USER"),
    'password': os.getenv("DB_PASSWORD"),
    'host': os.getenv("DB_HOST"),
    'port':os.getenv("DB_PORT")
}
insert_into_info_lab(df, connection_params)

Se insertaron 11795 filas en la tabla 'infolab'.


In [23]:
df_paciente = df[['identificacion', 'nombre1', 'sexo','id_paciente','tipo_de_identificacion', 'fecha_nacimiento']].copy()
df_paciente = df_paciente.drop_duplicates(subset=['id_paciente'], keep='first')
df_paciente['identificacion'] = df_paciente['identificacion'].astype(str).str.replace('.0', '', regex=False)
print(df_paciente.head())  # Mostrar las columnas del DataFrame transformado

  identificacion                           nombre1 sexo id_paciente  \
0       12865021         GOMEZ SALAS VICENTE RAMON    M  CC12865021   
1        5256941        ARTEAGA ROSERO JOSE MARINO    M   CC5256941   
2       27093153  DELGADO CABRERA ROMELIA TRINIDAD    F  CC27093153   
3       27387098  OBANDO ARTEAGA CENEIDA GUADALUPE    F  CC27387098   
4       27079284         BENAVIDES  LAURA VICTORIA    F  CC27079284   

  tipo_de_identificacion fecha_nacimiento  
0                     CC       1953-04-05  
1                     CC       1944-11-14  
2                     CC       1957-11-28  
3                     CC       1960-08-05  
4                     CC       1943-02-04  


In [24]:
def renombrar_campos_paciente(df_paciente):
    """
    Renombra los campos del DataFrame df_paciente a los nombres especificados.

    Parámetros:
    - df_paciente: DataFrame con los campos originales.

    Retorna:
    - DataFrame con los campos renombrados.
    """

    
    # Agregar columnas faltantes con valores nulos
    columnas_faltantes = [
        'nombre2', 'nombre3', 'apellido1', 'apellido2',
        'fecha_nacimiento', 'municipio_residencia', 'codigo_residencia',
        'barrio_residencia', 'direccion', 'edad', 'telefono'
    ]
    for columna in columnas_faltantes:
        if columna not in df_paciente.columns:
            df_paciente[columna] = None
    
    return df_paciente

In [25]:
df_paciente_renombrado = renombrar_campos_paciente(df_paciente)
print(df_paciente_renombrado.head())

  identificacion                           nombre1 sexo id_paciente  \
0       12865021         GOMEZ SALAS VICENTE RAMON    M  CC12865021   
1        5256941        ARTEAGA ROSERO JOSE MARINO    M   CC5256941   
2       27093153  DELGADO CABRERA ROMELIA TRINIDAD    F  CC27093153   
3       27387098  OBANDO ARTEAGA CENEIDA GUADALUPE    F  CC27387098   
4       27079284         BENAVIDES  LAURA VICTORIA    F  CC27079284   

  tipo_de_identificacion fecha_nacimiento nombre2 nombre3 apellido1 apellido2  \
0                     CC       1953-04-05    None    None      None      None   
1                     CC       1944-11-14    None    None      None      None   
2                     CC       1957-11-28    None    None      None      None   
3                     CC       1960-08-05    None    None      None      None   
4                     CC       1943-02-04    None    None      None      None   

  municipio_residencia codigo_residencia barrio_residencia direccion  edad  \
0       

In [26]:
cargar_dfpaciente(df_paciente_renombrado,'paciente', connection_params)

Insertados 1 registros nuevos en paciente.


(True, 1)